In [10]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [11]:
from azureml.core import Workspace

# Parameters (make sure these are correct)
subscr_id = "" 
resc_name = ""
worksp_name = ""

# Get a handle to the workspace
ml_client = MLClient(
    credential = credential,
    subscription_id = subscr_id,
    resource_group_name = resc_name,
    workspace_name = worksp_name,
)

# ws 
ws = Workspace(subscr_id, resc_name, worksp_name)

In [12]:
from azure.ai.ml.entities import AmlCompute

compute_target = "august-test-compute"

# let's see if the compute target exists
cluster = ml_client.compute.get(compute_target)
print(
    f"You have a cluster named {compute_target}, we'll use it as is."
)

# If you get the following error:
# 'ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials'
# You may need to authenticate the compute (somewhere above)

# If you want to create a compute from Python:

# try:
#     # let's see if the compute target already exists
#     cluster = ml_client.compute.get(compute_target)
#     print(
#         f"You already have a cluster named {compute_target}, we'll reuse it as is."
#     )

# except Exception:
#     print("Creating a new gpu compute target...")

    # # Let's create the Azure ML compute object with the intended parameters
    # cluster = AmlCompute(
    #     # Name assigned to the compute cluster
    #     name="cpu-cluster",
    #     # Azure ML Compute is the on-demand VM service
    #     type="amlcompute",
    #     # VM Family
    #     size="Standard_E4ds_v4",
    #     # Minimum running nodes when there is no job running
    #     min_instances=0,
    #     # Nodes in cluster
    #     max_instances=4,
    #     # How many seconds will the node running after the job termination
    #     idle_time_before_scale_down=180,
    #     # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
    #     tier="Dedicated",
    # )

    # # Now, we pass the object to MLClient's create_or_update method
    # cluster = ml_client.begin_create_or_update(cluster).result()

print(
    f"AMLCompute with variable name 'compute_target', the compute size is {cluster.size}"
)

You have a cluster named august-test-compute, we'll use it as is.
AMLCompute with variable name 'compute_target', the compute size is STANDARD_DS11_V2


In [13]:
# For job
env_name = "test4-environment@latest"

# Or
from azureml.core import Environment
mycustomenv = Environment(name="test4-environment@latest")

In [14]:
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.identity import DefaultAzureCredential

# Load data asset
train_data_asset = ml_client.data.get("test_dataasset", version="1") # Split into train and vali in script

In [15]:
from azure.ai.ml import command
from azure.ai.ml import Input
import os
import json

# Pass hyperparameter values
param = {
    "iterations": 30,
    "learning_rate": 0.00025,
    "img_per_batch": 3, # the real "batch size"
    "roi_batch_size": 16 # the ROI head "batch size"
}

# Serialize the dictionary to a JSON file
with open('param.json', 'w') as json_file:
    json.dump(param, json_file)

# Read the contents of param.json to set description of job
with open('param.json', 'r') as json_file:
    description = json.load(json_file)


In [16]:
# Set job and run
job = command(
     code=".", 
     command='python detectron2_train.py --traindata ${{inputs.train_data}}',
     inputs={
         "train_data": Input(path=train_data_asset.id,
             type=AssetTypes.URI_FOLDER,
             mode=InputOutputModes.RO_MOUNT # Mount (.RO_MOUNT) or download (.DOWNLOAD)
         ) 
     },
     environment=env_name,
     outputs={},  # Add any required outputs here
     description=description
 )

ml_client.jobs.create_or_update(job)

Uploading august.slomp (0.1 MBs): 100%|██████████| 100406/100406 [00:00<00:00, 683601.34it/s]




Experiment,Name,Type,Status,Details Page
august,nifty_zebra_8pg3lfx0x5,command,Starting,Link to Azure Machine Learning studio


In [ ]:
from azure.ai.ml.sweep import Uniform

# we will reuse the command_job created before. we call it as a function so that we can apply inputs
job_for_sweep = job(
    learning_rate=Uniform(min_value=0.0005, max_value=0.005),
    momentum=Uniform(min_value=0.9, max_value=0.99),
)